In [1]:
# System imports
from datetime import datetime
import numpy as np
import pandas as pd

# Third-party imports
import yfinance as yf
from pathlib import Path

# Local imports
from portfolio.holdings import Portfolio
from portfolio.commsec import Trades
import datehandler

# Vars
today = datetime.today().date()

In [2]:
trades = Trades()
trades_df = trades.all

p_dates = datehandler.date_list(trades_df.index[-1], today)  # Portfolio dates index

tickers = list(sorted(set(trades_df.Ticker.to_list())))
props = ['TradeType','TradeVolume','TradePrice','HoldingVolume','Vwap']

columns = pd.MultiIndex.from_product([tickers,props], names=['Tickers','Props'])
df_p = pd.DataFrame(None, index=pd.DatetimeIndex(p_dates), columns=columns)  # Create multiindex portfolio df

# Clean index and columns
df_p.index.name = 'Date'
df_p = df_p.reindex(sorted(df_p.columns),axis=1)  # Sort columns
df_p = df_p.reindex(sorted(df_p.index),axis=0)  # Sort index
df_p

Tickers              A2M                                                 ACW  \
Props      HoldingVolume TradePrice TradeType TradeVolume Vwap HoldingVolume   
Date                                                                           
2015-01-07           NaN        NaN       NaN         NaN  NaN           NaN   
2015-01-08           NaN        NaN       NaN         NaN  NaN           NaN   
2015-01-09           NaN        NaN       NaN         NaN  NaN           NaN   
2015-01-12           NaN        NaN       NaN         NaN  NaN           NaN   
2015-01-13           NaN        NaN       NaN         NaN  NaN           NaN   
...                  ...        ...       ...         ...  ...           ...   
2020-01-20           NaN        NaN       NaN         NaN  NaN           NaN   
2020-01-21           NaN        NaN       NaN         NaN  NaN           NaN   
2020-01-22           NaN        NaN       NaN         NaN  NaN           NaN   
2020-01-23           NaN        NaN       NaN         NaN  NaN           NaN   
2020-01-24           NaN        NaN       NaN         NaN  NaN           NaN   

Tickers                                           ...           WZR  \
Props      TradePrice TradeType TradeVolume Vwap  ... HoldingVolume   
Date                                              ...                 
2015-01-07        NaN       NaN         NaN  NaN  ...           NaN   
2015-01-08        NaN       NaN         NaN  NaN  ...           NaN   
2015-01-09        NaN       NaN         NaN  NaN  ...           NaN   
2015-01-12        NaN       NaN         NaN  NaN  ...           NaN   
2015-01-13        NaN       NaN         NaN  NaN  ...           NaN   
...               ...       ...         ...  ...  ...           ...   
2020-01-20        NaN       NaN         NaN  NaN  ...           NaN   
2020-01-21        NaN       NaN         NaN  NaN  ...           NaN   
2020-01-22        NaN       NaN         NaN  NaN  ...           NaN   
2020-01-23        NaN       NaN         NaN  NaN  ...           NaN   
2020-01-24        NaN       NaN         NaN  NaN  ...           NaN   

Tickers                                                    ZNO             \
Props      TradePrice TradeType TradeVolume Vwap HoldingVolume TradePrice   
Date                                                                        
2015-01-07        NaN       NaN         NaN  NaN           NaN        NaN   
2015-01-08        NaN       NaN         NaN  NaN           NaN        NaN   
2015-01-09        NaN       NaN         NaN  NaN           NaN        NaN   
2015-01-12        NaN       NaN         NaN  NaN           NaN        NaN   
2015-01-13        NaN       NaN         NaN  NaN           NaN        NaN   
...               ...       ...         ...  ...           ...        ...   
2020-01-20        NaN       NaN         NaN  NaN           NaN        NaN   
2020-01-21        NaN       NaN         NaN  NaN           NaN        NaN   
2020-01-22        NaN       NaN         NaN  NaN           NaN        NaN   
2020-01-23        NaN       NaN         NaN  NaN           NaN        NaN   
2020-01-24        NaN       NaN         NaN  NaN           NaN        NaN   

Tickers                                
Props      TradeType TradeVolume Vwap  
Date                                   
2015-01-07       NaN         NaN  NaN  
2015-01-08       NaN         NaN  NaN  
2015-01-09       NaN         NaN  NaN  
2015-01-12       NaN         NaN  NaN  
2015-01-13       NaN         NaN  NaN  
...              ...         ...  ...  
2020-01-20       NaN         NaN  NaN  
2020-01-21       NaN         NaN  NaN  
2020-01-22       NaN         NaN  NaN  
2020-01-23       NaN         NaN  NaN  
2020-01-24       NaN         NaN  NaN  

[1318 rows x 175 columns]

In [3]:
dropped_cols = ['Market','Ticker','TradePrice','Brokerage']
for ticker in tickers:
    # Add trade dataframe values to portfolio dataframe
    df_t = trades_df[trades_df.Ticker == ticker].copy()
    df_t.loc[df_t['TradeType'] == 'S', 'Volume'] *= -1  # Don't worry about the warning
    df_t = df_t.drop(columns=dropped_cols)  # Drop columns first
    df_t = df_t.rename(columns={            # Then rename columns - due to conflicting name!
        'Volume':'TradeVolume',
        'EffectivePrice':'TradePrice',
        })
    df_p[ticker] = df_t

In [4]:
# Lookup prices
# Build list of tickers for yfinance
lookup_tickers = [f'{ticker}.AX' for ticker in df_p.columns.levels[0].to_list()]
lookup_tickers = ' '.join(lookup_tickers)

prices = yf.download(lookup_tickers, start=p_dates[0], end=p_dates[-1])

[*********************100%***********************]  35 of 35 completed

2 Failed downloads:
- IBXO.AX: No data found for this date range, symbol may be delisted
- SRS.AX: No data found, symbol may be delisted


In [5]:
tickers_with_prices = prices.columns.levels[1].to_list()
tickers_with_prices = [ticker.replace('.AX','') for ticker in tickers_with_prices]  

prices.columns.set_levels(tickers_with_prices, level=1, inplace=True)

In [88]:
def _build_from_trades(df):
    df_ticker = df.copy()

    df_ticker['HoldingVolume'] = df_ticker['TradeVolume']

    # Vwap: BookValue / HoldingVolume
    df_ticker.loc[df_ticker['TradeType'] == 'B','TradeEncoding'] = 1
    df_ticker.loc[df_ticker['TradeType'] == 'S','TradeEncoding'] = 0
    df_ticker['BuyValue'] = df_ticker['TradeVolume'] * df_ticker['TradePrice'] * df_ticker['TradeEncoding']
    df_ticker['BuyVolume'] = df_ticker['TradeVolume'] * df_ticker['TradeEncoding']
    df_ticker['BuyVolume'] = df_ticker['BuyVolume'].replace(0,np.nan)  # Temp calc column
    df_ticker['BuyValue'] = df_ticker['BuyValue'].replace(0,np.nan)  # Temp calc column
    df_ticker['Vwap'] = df_ticker['BuyValue'].divide(df_ticker['BuyVolume'])

    df_ticker = df_ticker.drop(columns=['BuyVolume','BuyValue','TradeEncoding'])  # Remove temp calc columns
    
    return df_ticker


# Build portfolio
for count, ticker in enumerate(df_p.columns.levels[0]):
    print(f'\r{ticker} | Progress {count+1}/{len(df_p.columns.levels[0])} ',end='',flush=True)
    if ticker not in tickers_with_prices:
        print(f'Skipping {ticker} due to no price data...')
        continue
    
    df_p[ticker] = _build_from_trades(df_p[ticker])     # Build porfolio from trades
    df_p[ticker, 'ClosePrice'] = prices['Close'][ticker]  # Load prices into dataframe

    try:  # Dividends and stock splits
        stock = yf.Ticker(f'{ticker}.AX')
        df_actions = stock.actions
    except TypeError:
        print(f'\rSomething went wrong!',flush=True)
        pass
    else:
        if len(df_actions[df_actions['Dividends'] > 0]) > 0:
            df_p[ticker, 'Dividends'] = df_actions['Dividends'].replace(0,np.nan)
        if len(df_actions[df_actions['Stock Splits'] > 0]) > 0:
            df_p[ticker, 'StockSplits'] = df_actions['Stock Splits'].replace(0,np.nan)
            df_p[ticker, 'HoldingVolume'] = df_p[ticker, 'HoldingVolume'] * df_p[ticker, 'StockSplits']
            df_p[ticker, 'Vwap'] = df_p[ticker, 'Vwap'].divide(df_p[ticker, 'StockSplits'])
    df_p[ticker,'Vwap'] = df_p[ticker,'Vwap'].fillna(method='ffill')
    df_p[ticker,'HoldingVolume'] = df_p[ticker,'TradeVolume'].fillna(0).cumsum()

    # Calculating returns
    df_p[ticker, 'Cashflow'] = df_p[ticker, 'TradePrice'] * df_p[ticker, 'TradeVolume']
    df_p[ticker, 'Cashflow'] = df_p[ticker, 'Cashflow'].fillna(0)
    
    df_p[ticker, 'EndValue'] = df_p[ticker, 'HoldingVolume'] * df_p[ticker, 'ClosePrice'].fillna(method='ffill')
    df_p[ticker, 'StartValue'] = df_p[ticker, 'EndValue'].shift(1).replace(0,np.nan)
    df_p[ticker, 'DailyReturn'] = df_p[ticker, 'EndValue'].divide(
        df_p[ticker, 'StartValue'].add(df_p[ticker, 'Cashflow'])
        ) - 1
    df_p[ticker, 'DailyReturn'] = df_p[ticker, 'DailyReturn'].replace(-1,np.nan) + 1
    df_p[ticker, 'DailyReturn'] = df_p[ticker, 'DailyReturn'].cumprod()

    
# df_p = df_p.drop(['EndValue','StartValue','Cashflow'], axis='columns',level=1)
df_p[('portfolio', 'EndValue')] = df_p.xs('EndValue', level='Props', axis=1).sum(axis=1, min_count=1)
df_p[('portfolio', 'StartValue')] = df_p.xs('StartValue', level='Props', axis=1).sum(axis=1, min_count=1)
df_p[('portfolio', 'Cashflow')] = df_p.xs('Cashflow', level='Props', axis=1).sum(axis=1, min_count=1)
df_p[('portfolio','DailyReturn')] = ((df_p[('portfolio','EndValue')] -
                                        (df_p[('portfolio','StartValue')]+df_p[('portfolio','Cashflow')])
                                    )
                                    /
                                    (df_p[('portfolio','StartValue')]+df_p[('portfolio','Cashflow')])
                                    ) + 1
df_p[('portfolio','DailyReturn')] = df_p[('portfolio','DailyReturn')].cumprod()

IBXO | Progress 10/36- IBXO.AX: 1d data not available for startTime=-2208988800 and endTime=1579872890. Only 100 years worth of day granularity data are allowed to be fetched per request.
Something went wrong!
SRS | Progress 30/36- SRS.AX: No data found, symbol may be delisted
Something went wrong!
portfolio | Progress 36/36Skipping portfolio due to no price data...


In [89]:
df_p.MDR

Props,HoldingVolume,TradePrice,TradeType,TradeVolume,Vwap,ClosePrice,DailyReturn,Cashflow,EndValue,StartValue,StockSplits
Date,,,,,,,,,,,
2015-01-07,0.0,NaN,NaN,NaN,NaN,0.140,NaN,0.0,0.000000,NaN,NaN
2015-01-08,0.0,NaN,NaN,NaN,NaN,0.084,NaN,0.0,0.000000,NaN,NaN
2015-01-09,0.0,NaN,NaN,NaN,NaN,0.084,NaN,0.0,0.000000,NaN,NaN
2015-01-12,0.0,NaN,NaN,NaN,NaN,0.084,NaN,0.0,0.000000,NaN,NaN
2015-01-13,0.0,NaN,NaN,NaN,NaN,0.084,NaN,0.0,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2020-01-20,18886.0,NaN,NaN,NaN,NaN,0.570,1.536388,0.0,10765.019865,10009.579460,NaN
2020-01-21,18886.0,NaN,NaN,NaN,NaN,0.570,1.536388,0.0,10765.019865,10765.019865,NaN
2020-01-22,18886.0,NaN,NaN,NaN,NaN,0.560,1.509434,0.0,10576.160045,10765.019865,NaN


In [38]:
test = df_p['RMD'].DailyReturn + 1
test = test.cumprod()
# test[test.isna() == False]
test

Date
2015-01-07         NaN
2015-01-08         NaN
2015-01-09         NaN
2015-01-12         NaN
2015-01-13         NaN
                ...   
2020-01-20    3.151072
2020-01-21    3.168782
2020-01-22    3.238261
2020-01-23    3.257334
2020-01-24    3.215102
Name: DailyReturn, Length: 1318, dtype: float64

In [46]:
'''
The problem is: when I sell out of a position, then buy back in, the vwap is reset.
'''

df_p.RMD.Vwap.unique()

array([nan, 7.5665, 10.330584795321638], dtype=object)

In [47]:
df_p.RMD[df_p.RMD.Vwap == 10.330584795321638]

Props,HoldingVolume,TradePrice,TradeType,TradeVolume,Vwap,ClosePrice,Cashflow,EndValue,StartValue,DailyReturn
Date,,,,,,,,,,
2018-07-03,213,14.2237,B,213,10.3306,14.000000,3029.64,2982,NaN,NaN
2018-07-04,213,NaN,NaN,NaN,10.3306,13.980000,0.00,2977.74,2982.000000,-0.001429
2018-07-05,213,NaN,NaN,NaN,10.3306,14.070000,0.00,2996.91,2977.739902,0.006438
2018-07-06,213,NaN,NaN,NaN,10.3306,14.360000,0.00,3058.68,2996.909935,0.020611
2018-07-09,213,NaN,NaN,NaN,10.3306,14.240000,0.00,3033.12,3058.679927,-0.008357
...,...,...,...,...,...,...,...,...,...,...
2020-01-20,213,NaN,NaN,NaN,10.3306,23.129999,0.00,4926.69,4954.380049,-0.005589
2020-01-21,213,NaN,NaN,NaN,10.3306,23.260000,0.00,4954.38,4926.689821,0.005620
2020-01-22,213,NaN,NaN,NaN,10.3306,23.770000,0.00,5063.01,4954.380049,0.021926


In [48]:
df_p.RMD[df_p.RMD.TradeType == 'S']

Props,HoldingVolume,TradePrice,TradeType,TradeVolume,Vwap,ClosePrice,Cashflow,EndValue,StartValue,DailyReturn
Date,,,,,,,,,,
2018-05-15,227,13.423,S,-73,7.5665,13.55,-979.88,3075.85,4070.999908,-0.00494
2018-06-21,0,14.5221,S,-227,7.5665,14.58,-3296.52,0,3280.149957,NaN


In [49]:
df_p.loc['2018-06-18':'2018-06-25'].RMD

Props,HoldingVolume,TradePrice,TradeType,TradeVolume,Vwap,ClosePrice,Cashflow,EndValue,StartValue,DailyReturn
Date,,,,,,,,,,
2018-06-18,227,NaN,NaN,NaN,7.5665,14.17,0.00,3216.59,3248.370095,-0.009783
2018-06-19,227,NaN,NaN,NaN,7.5665,14.28,0.00,3241.56,3216.590017,0.007763
2018-06-20,227,NaN,NaN,NaN,7.5665,14.45,0.00,3280.15,3241.559939,0.011905
2018-06-21,0,14.5221,S,-227,7.5665,14.58,-3296.52,0,3280.149957,NaN
2018-06-22,0,NaN,NaN,NaN,7.5665,14.48,0.00,0,NaN,NaN
2018-06-25,0,NaN,NaN,NaN,7.5665,14.49,0.00,0,NaN,NaN
